# FEO API

*To use this notebook you must have a (free) FEO account.*

This notebook guides the user through making calls directly to the FEO-API using Python's requests module.

Tooling to generate request headers:

In [ ]:
import time

import requests

AUTH0_CLIENT_ID = "HhT6aGS8u3Pg4PkVQ8sKUtnrtg0x7nUk"
AUTH0_DOMAIN = "prod-feo-tz.eu.auth0.com"
BASE_URL = "https://api.feo.transitionzero.org"


def get_headers(config=None):
    device_code_payload = {
        "client_id": AUTH0_CLIENT_ID,
        "scope": "openid profile offline_access",
        "audience": BASE_URL,
    }

    device_code_response = requests.post(
        f"https://{AUTH0_DOMAIN}/oauth/device/code",
        json=device_code_payload,
        timeout=30,  # noqa
    )
    device_code_response.raise_for_status()

    if device_code_response.status_code != 200:
        raise ValueError("Error generating device code")

    device_code_data = device_code_response.json()
    print(
        "1. In a browser navigate to: ", device_code_data["verification_uri_complete"]
    )
    print("2. Enter the following code: ", device_code_data["user_code"])

    token_payload = {
        "grant_type": "urn:ietf:params:oauth:grant-type:device_code",
        "device_code": device_code_data["device_code"],
        "client_id": AUTH0_CLIENT_ID,
        "audience": BASE_URL,
    }

    authenticated = False
    print("Checking for authentication", end="")
    while not authenticated:
        token_response = requests.post(
            f"https://{AUTH0_DOMAIN}/oauth/token",
            data=token_payload,
            timeout=30,  # noqa
        )

        token_data = token_response.json()
        if token_response.status_code == 200:
            print("Authenticated!")
            print("- Id Token: {}...".format(token_data["id_token"][:10]))
            authenticated = True
        elif token_data["error"] not in ("authorization_pending", "slow_down"):
            print(token_data["error_description"])
            raise ValueError("Error generating device code")
        else:
            print(".", end="")
            time.sleep(device_code_data["interval"])

    token = token_data.get("access_token")
    headers = {"Authorization": f"Bearer {token}"}

    return headers

Get headers for authenticating requests to the API:

In [ ]:
headers = get_headers()

Send requests to the API using the Python requests module - endpoints and parameters are detailed in the FEO API docs [here](https://docs.feo.transitionzero.org/redoc)

In [ ]:
params = {"limit": 5}

response = requests.get(f"{BASE_URL}/latest/nodes", params=params, headers=headers)

response.json()